# ATE estimation (Single Stage)

## Main Idea
In the preliminary section, we introduced three categories of methods that have been widely used in ATE estimation, which are direct method (DM), importance sampling estimator (IS), and doubly robust estimator (DR).



## Algorithm Details


## Key Steps


## Demo Code
In the following, we exhibit how to apply the learner on real data to do policy learning and policy evaluation, respectively.

### 1. Direct Method

In [1]:
# import data


**Interpretation:**

### 2. Importance Sampling Estimator

### 3. Doubly Robust Estimator

## References
1. Murphy, S. A. (2005). A generalization error for Q-learning.
2. Song, R., Wang, W., Zeng, D., & Kosorok, M. R. (2015). Penalized q-learning for dynamic treatment regimens. Statistica Sinica, 25(3), 901.